<center><br><br>
    Arkansas Work-Based Learning to Workforce Outcomes <br>
    Applied Data Analytics Training | Spring 2022
    <h1> Characterizing Demand: Descriptive Analysis </h1>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Coleridge Initiative</a>
    </span>
    <center> Joshua Edelmann, Rukhshan Arif Mian, Benjamin Feder</center>
</center>

***

# Introduction

Characterizing the labor market demand can help us understand the different types of employers at a city, county, or state level. A majority of the research on labor market outcomes lays emphasis on the role of the employee (labor market supply). While this is important, understanding the employer's role is also critical for developing employment outcomes.

In the notebook `03B_Longitudinal_Analysis_Interactive_Guide.ipynb`, we used descriptive statistics to understand employment outcomes for our cohort. This allowed us to see the various patterns (for example: missingness) exhibited in the data that could impact any statistical analysis performed. The goal of this notebook is now to understand how we can use descriptive statistics for the purpose of characterizing labor demand. This will allow us to understand the types of employers individuals in our cohort are employed by and how this can impact employment outcomes. More specifically, we can identify employer characteristics that are associated with positive employment outcomes for our cohort. 

In [ ]:
# Switching off warnings
options(warn = -1)

# Database interaction imports
suppressMessages(library(odbc))

# data manipulation/visualization
suppressMessages(library(tidyverse))

# scaling data, calculating percentages, overriding default graphing
suppressMessages(library(scales))

# for as.yearqtr()
suppressMessages(library(zoo))

#Switching on warnings
options(warn = 0)

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

# Data Sources

We utilize the following data sources throughout this notebook:

- **Employer Measures** (`employer_yearly_agg`): Measures we created in the `Supplemental_Employer_Measures.ipynb` notebook
- **Apprenticeship Completer Cohort** (`nb_cohort`): Completers that finished their training in 2015. We created a larger version of this cohort including those who completed in 2016 and 2017 in the `02_Creating_a_cohort.ipynb` notebook from Module 2. 
- **UI Wages** (`ui_wages_lehd`): UI Wage records for Arkansas

## Describing Employer Measures

This notebook will focus on 3 essential categories of annualized employer-level measures when it comes to characterizing labor market demand: Firm characteristics, Opportunity, and Stability. We define the specific employer measures that fall into each category below:

**Firm characteristics**:

- Total Payroll
- Average earnings per employee 
- Average full quarter earnings per employee
- Earnings per employee at 25th percentile
- Earnings per employee at 75th percentile
- Total full quarter employment
- Total employment

**Opportunity**:

- Number of new hires
- Employment growth rate
- Hiring growth rate, with growth rate as

 $$ \ \large{g_{et}=\frac{(x_{et} - x_{e,t-1})}{\frac{(x_{et} + x_{e,t-1})}{2}}} $$

where: 
 - $x_{et}$:  Total Employment/Hiring/Separation at time *t*
 - $x_{et-1}$:  Total Employment/Hiring/Separation at time *t-1*
 


**Stability**:
- Separation growth rate (this is calculated using the same formula as above)
- Number of new hires who become full quarter employees (hired in t-1 whom we see in t+1)
- Ratio of full quarter employees to total number of employees 


As a reminder, these measures were created using Arkansas' UI Wage Records and filtered out any employers that had < 5 employees. 


To learn more about what each of these measures means and how they were created, we encourage you to look at the supplementary worksheet (`Supplemental_Employer_Measure_Worksheet.xlsx`). 

As discussed in the introduction, we can use descriptive analysis to understand the correlation between employer measures and employment outcomes. We can use these to address questions such as, "for apprenticeship completers, by what types of employers are they being employed? Are these high-growth or low-growth employers?"

In this notebook, we focus on single measures from 2 of the 3 employer measure categories: 
1. Total Employment (Firm Characteristics)
2. Employment Growth Rate (Opportunity)

Before diving into these measures, we look at how the **employer_yearly_agg** table is structured. 

In [ ]:
qry <- "
SELECT * 
FROM tr_ar_2022.dbo.employer_yearly_agg
"

employer_measures <- dbGetQuery(con, qry)

head(employer_measures)

Let's look at the columns in the **employer_measures** data frame.

In [ ]:
names(employer_measures)

As a reminder, each observation in this data frame corresponds to an employer-year combination. We have data for all years that fall in the following range:

In [ ]:
# range calculates the min and max values for a variable of interest
range(employer_measures$year) 

One important note here is that not all employers appear in this data, and those who appear may not appear every year. There are several reasons this could happen:

1. A non-UI covered employer will not submit wages to UI. For example: federal work or gig work.
2. The employer may have less than 5 employees (these were filtered out when constructing the employer measures).
3. The employer may not yet be established or has gone out of business.

Keep all of this in mind when considering the inferential limitations when using the **employer_measures** table.

# Analyzing Firm Characteristics

In this section, we use firm characteristics to understand how we can characterize labor demand for our cohort of apprenticeship completers. 

We explore the following research question: 
-  Do apprenticeship completers from our cohort stay longer at a job when they are employed  by larger employers compared to smaller employers? 

We define larger employers as those who have an above average number of employees. Similarly, we define small employers as those who have a below average number of employees. We will be looking at summary statistics for all employers and for employers who employed apprenticeship completers from our cohort during the quarter of their completion. If they had multiple employers, we will then look at the relationship between employer characteristics and employee outcomes.

In this analysis, we will include individuals from our cohort who were employed in the quarter in which they completed their apprenticeship (of just those who completed in 2015). We use the quarter of completion to look at how the characteristics of apprenticeship completers' initial employers impact their outcomes. From there, we will then count the number of quarters they worked for the employer in which they were employed by at the time of completion.

The first step in approaching this research question is linking our cohort with the UI Wage data to identify the primary employers at the time of completion for apprenticeship completers in 2015. 

> Note: We consider employer measures from 2014 since this would be the information that was available to apprentices prior to completion in 2015. It is very unlikely they were used in practice; however, we hope to provide a practical tool that can be utilized in the future. Additionally, by employer, we only focus on the primary employer for the individual in the quarter, which we define as the employer in which they earned the most in that time frame.

## Linking Cohort to Wages 

We can leverage similar code to that used in `03B_Longitudinal_Analysis_Interactive_Guide.ipynb`, which uses the dimensional model. Eventually, we will look at employer retention for the next four quarters post graduation, so we will query all wage records between the quarter of completion and three subsequent quarters.

In [ ]:
# using dimensional model to get primary employer information
qry <- "
SELECT
F.Quarter_ID - P.Apprenticeship_End_Quarter_ID AS Quarters_Relative_to_Completion,
P.Person_ID,
F.Primary_Employer_Wages ,
PE.Federal_EIN
FROM 
tr_ar_2022.dbo.nb_cohort C --COHORT
JOIN tr_ar_2022.dbo.AR_MDIM_Person P ON (P.Apprentice_Number=C.apprnumber) --PERSON
JOIN tr_ar_2022.dbo.AR_FACT_Quarterly_Observation F --QUARTERLY OBSERVATION FACT
	ON (P.Person_ID=F.Person_ID) 
	AND (F.Quarter_ID BETWEEN (P.Apprenticeship_End_Quarter_ID) AND (P.Apprenticeship_End_Quarter_ID+4))  --QTRS POST COMPLETION
JOIN tr_ar_2022.dbo.AR_RDIM_NAICS_National_Industry NNI ON (P.Apprenticeship_NAICS_National_Industry_ID=NNI.NAICS_National_Industry_ID) --APPRENTICESHIP INDUSTRY
JOIN tr_ar_2022.dbo.AR_MDIM_Employer PE ON (PE.Employer_ID=F.Primary_Employer_ID)  --PRIMARY EMPLOYER
WHERE P.Apprenticeship_Completer='Y' and YEAR(C.exitwagedt) = 2015
"

cohort_wages_empr <- dbGetQuery(con, qry)

head(cohort_wages_empr)

Count the unique number of individuals from our cohort who were employed along with the unique number of employers by whom they were employed at the time of completion.

In [ ]:
# save data frame of employment info at time of completion
cohort_wages_empr_comp <- cohort_wages_empr %>% filter(Quarters_Relative_to_Completion == 0)

# see summary stats
cohort_wages_empr_comp %>% 
    summarize(unique_indiv = n_distinct(Person_ID),
              unique_empr = n_distinct(Federal_EIN))

## Extracting Employer Measures

Now that we have information on the employers who employed individuals from our apprenticeship completers cohort at the time of their completion, we can create a subset of the **employer_measures** data frame by keeping unique employers from the previous step who have records in 2014.

In [ ]:
employer_measures_cohort <- employer_measures %>%
    # filtering on employer number and year
    filter(federal_ein %in% cohort_wages_empr_comp$Federal_EIN, year == 2014)

In [ ]:
# Count the number of unique employers in this new data frame
employer_measures_cohort %>% 
    summarize(unique_emp = n_distinct(federal_ein))

Note that the number of employers falls when we apply this filter. Recall that measures were not developed for firms that had fewer than 5 employees and if the UI Wage Records did not exist for these firms in 2014.

## Descriptive Statistics
The measure of interest for this section is the average number of individuals working for an employer in a quarter, (averaged over the year) **avg_num_employed**. 
> If employer A employed 100 people in 2014Q1, 102 people in 2014Q2, 98 people in 2014Q3 and 104 people in 2014Q4, their **avg_num_employed** in 2014 would be 101. 

As a first step, we will calculate summary statistics for this variable only looking at employers who employed individuals from our cohort during the quarter of their apprenticeship completion. Then we will compare these results with summary statistics for all other employers (who did not employ individuals from our cohort) in 2014.

#### Employers who employed apprenticeship completers from our cohort at the time of completion

In [ ]:
employer_measures_cohort %>%        
    summarize(
        # mean
        mean_emp = mean(avg_num_employed),
        # median
        median_emp = median(avg_num_employed), 
        # standard deviation
        sd_emp = sd(avg_num_employed),
        # min value
        min_emp = min(avg_num_employed), 
        # max value
        max_emp = max(avg_num_employed)
    )

#### Employers who did not employ apprenticeship completers from our cohort at the time of completion

In [ ]:
employer_measures %>%
    # filtering out employers that hired from our cohort
    filter(!(federal_ein %in% cohort_wages_empr_comp$Federal_EIN), 
    # keeping year == 2014
           year == 2014) %>%
    summarize(mean_emp = mean(avg_num_employed), 
             median_emp = median(avg_num_employed), 
             sd_emp = sd(avg_num_employed),
             min_emp = min(avg_num_employed), 
             max_emp = max(avg_num_employed))


We see how both the mean and median values for average number of employees are REDACTED for employers who employed at least one individual from our cohort at the time of apprenticeship completion compared to all other employers in 2014. It seems as though apprenticeship completers are REDACTED likely to be hired by larger employers.

## Differentiating Employers

We use the mean value for `avg_num_employed` to create a categorical variable to differentiate employers into 2 groups:

1. Large: Employers with above-average number of employees
2. Small: Employers with below-average number of employees

#### Establish groups of employers

We use the employer measures to establish the average number of employees across all employers in 2014. 

In [ ]:
# extract mean value for avg_num_employed 
mean_emp <- employer_measures %>% 
    filter(year == 2014) %>%
    summarize(mean_emp = mean(avg_num_employed))

mean_emp

In [ ]:
# creating a variable: emp_cat that equals Large if avg_num_employed >= mean
employer_measures <- employer_measures %>%
    filter(year == 2014) %>%
    mutate(emp_cat = ifelse(avg_num_employed >= mean_emp$mean_emp, "Large", "Small"))

head(employer_measures)

In [ ]:
# check distribution of emp_cat
table(employer_measures$emp_cat)

Now that we have a categorical variable for the size of the employer, we can link **employer_measures_cohort** to our joined cohort-wages data frame for just the quarter of completion **cohort_wages_empr_comp**.

In [ ]:
employer_measures_cat <- employer_measures %>%
    # selecting out Employer number and categorical variable of interest
    select(federal_ein, emp_cat)

# performing an inner join so that we only keep individuals who are employed by employers for whom we have employer measures in 2014
cohort_wages_merged_measures <- inner_join(cohort_wages_empr_comp, 
          employer_measures_cat, 
          by=c("Federal_EIN" = "federal_ein"))

table(cohort_wages_merged_measures$emp_cat)

Now, we will count the number of unique individuals and employers we have in our merged dataframe.  

> Note: We expect this number to be lower than what we saw for **cohort_wages_empr** as not all employers are captured in **employer_measures** (reminder: employer may not exist or has fewer than 5).

In [ ]:
cohort_wages_merged_measures %>%
    summarize(unique_indiv = n_distinct(Person_ID), 
              unique_emp = n_distinct(Federal_EIN))

## Employee-primary employer combinations (matches)
Next, we select out the columns for **Person_ID**, **Federal_EIN** and **emp_cat** to have a data frame that corresponds to employee-primary employer matches in the quarter of apprenticeship completion.

In [ ]:
emp_empr_matches <- cohort_wages_merged_measures %>% 
    select(Person_ID, Federal_EIN, emp_cat) 

head(emp_empr_matches)

The next step in our methodology is to **count the number of quarters during the first year after completion** for which an employee had the same primary employer. 

## Quarters with same primary employer

The **emp_empr_matches** data frame contains a list of unique employee-primary employer combinations at the time of apprenticeship completion, and **cohort_wages_empr** contains employee-primary employer combinations for 4 quarters after completion. To identify the number of quarters each individual had the same primary employer, we will merge the 2 data frames and count the unique number of quarters for each employee-employer pair. 

In [ ]:
merged_cohort <- inner_join(emp_empr_matches, # employee-employer matches 
                            cohort_wages_empr, # employee-employer matches for first year after completion
                            by=c("Person_ID", "Federal_EIN"))

head(merged_cohort)

With the merged data frame, we can group by our variables of interest and count the number of distinct quarters after completion.

> Note: Since we are counting the first quarter as the quarter of graduation, we need to subtract 1 from `n_distinct(Quarters_Relative_to_Completion)`.

In [ ]:
merged_cohort_num_quart <- merged_cohort %>%
        group_by(Person_ID, Federal_EIN, emp_cat) %>%
        summarize(num_quart_emp = n_distinct(Quarters_Relative_to_Completion)-1) %>%
        ungroup()

head(merged_cohort_num_quart)

#### Portion of employees with the same primary employer by quarter

The goal here is to explore whether apprenticeship completers who are have their primary employment initially at a larger firm are more likely to stay with that primary employer compared to those who have their primary employment initially at a smaller one. To do this, we will calculate the portion of employees had the same primary employer in the four quarters after completion broken down by whether their primary employment was at a larger or smaller firm at the time of apprenticeship completion.

In [ ]:
# find proportion by size of firm
grouped <- merged_cohort_num_quart %>%
# can use count since there is one primary employer/employee combination per row
    count(emp_cat, num_quart_emp) %>%
    group_by(emp_cat) %>%
    mutate(
        prop = n/sum(n)
    )

grouped

## Visualizing Results

We visualize our results in the form of a bar plot to better understand outcomes. 

In [ ]:
ggplot(grouped, aes(x=num_quart_emp, y=prop, fill=emp_cat))+
  geom_bar(stat='identity', position='dodge')

As you can see from this visual: 
1. The most common outcome is REDACTED with the same primary employer for 4 quarters regardless of employer size. 
2. Apprenticeship completers initially primarily employed by smaller firms are REDACTED likely to be primarily employed by the same firm in the four subsequent quarters compared to their counterparts initially primarily employed at larger firms. 

# Opportunity

The second measure of interest for this notebook is Employment Growth Rate. The goal is to look at average employment growth rate and identify the outcomes of individuals who are initially primarily employed by high-, medium- and low-growth employers. 

The outcome we consider is the percent of individuals within each group that were employed in each quarter of the year following completion. For example, of our cohort that completed their apprenticeship in 2015 and were initially primarily employed by high-growth employers, what percentage was employed in the second, third and fourth quarters after graduation?

> Note: We are looking at employment in general and not just employment with the same primary employer. The underlying assumption that if an apprenticeship completer gets a "good" job, they might be able to transition into an even better job more easily. This allows us to understand a completer's total employment trajectory based on the type of primary employer they were employed by as soon as they finish. 

## Descriptive Statistics

Similar to what we did for Firm Characteristics, we perform basic summary statistics. Our measure of interest for this section is Employment Growth Rate which is captured by the following variable: **avg_emp_rate**. 


> Note: The values for **avg_emp_rate** are bound between -2 and 2.

In [ ]:
# getting the mean, median, standard deviation, min and max values for `avg_emp_rate`
employer_measures_cohort %>%
    summarize(mean_emp = mean(avg_emp_rate), 
             median_emp = median(avg_emp_rate), 
             sd_emp = sd(avg_emp_rate), 
             min_emp = min(avg_emp_rate), 
             max_emp = max(avg_emp_rate)
    )

Next, we compare our results with the all other employers from 2014.

In [ ]:
employer_measures %>%
    # filtering out employers that hired from our cohort
    filter(!(federal_ein %in% cohort_wages_empr_comp$Federal_EIN), 
    # keeping year == 2014
           year == 2014) %>%
    summarize(
        mean_emp = mean(avg_emp_rate), 
        median_emp = median(avg_emp_rate), 
        sd_emp = sd(avg_emp_rate), 
        min_emp = min(avg_emp_rate), 
        max_emp = max(avg_emp_rate)
    ) 


Employers who initially primarily employer members of our cohort of apprenticeship completers have REDACTED mean, but REDACTED median employment growth rates. 

> Note: This may indicate a difference in the overall shape of the distribution of employment rates between these two groups.

## Differentiating Employers
In this section, we are looking to identify high-, medium-, and low-growth employers. We define these as follows:

- High: >= 75th percentile of **avg_emp_rate**
- Medium: > 25th percentile and < 75th percentile of **avg_emp_rate**
- Low: <= 25th percentile of **avg_emp_rate**

We use R's **quantile** function to get the 25th and 75th percentiles for **avg_emp_rate** and assign these to **p25** and **p75**, respectively. 

In [ ]:
# use pull() to isolate the variable of interest in a vector
p <- employer_measures %>%
    filter(year == 2014) %>%
    pull(avg_emp_rate) %>%
    quantile(probs = c(.25, .75))

p

The first value corresponds to 25th percentile and the second value corresponds to the 75th percentile. 

In [ ]:
# extracting 25th and 75th percentile 
p25 <- p[1]
p75 <- p[2]

We define a categorical variable, **emp_rate_cat**, by taking into account the 25th and 75th percentiles. 

In [ ]:
employer_measures <- employer_measures %>%
    filter(year == 2014) %>%
    mutate(emp_rate_cat = case_when(
                                # Low: <= 25th percentile
                                avg_emp_rate <= p25 ~ "Low", 
                                # Medium: > 25th and < 75th percentile
                                avg_emp_rate > p25 & avg_emp_rate < p75 ~ "Medium", 
                                # High: >= 75th percentile
                                TRUE ~ "High")
          )

table(employer_measures$emp_rate_cat)

Next, we select columns for **federal_ein** and **emp_rate_cat** – this removes columns that we do not require for further analysis. 

In [ ]:
employer_measures_growth_cat <- employer_measures %>%
    filter(year==2014) %>%
    select(federal_ein, emp_rate_cat)

head(employer_measures_growth_cat)

## Linking with Cohort

Now that we have a categorical variable of our interest, we can link **employer_measures_growth_cat** to **cohort_wages_empr_comp** to add the categorical variable for employment growth rate to our primary employer-employee data at the time of apprenticeship completion. 

The resulting data frame will give us the growth category (High/Medium/Low) a apprenticeship completer's first employer falls in. 

> Note: We will only need the **Person_ID** and **emp_rate_cat** variables moving forward, so we will explicitly `select()` them after performing the join.

In [ ]:
cohort_wages_merged_growth_cat <- inner_join(cohort_wages_empr_comp, employer_measures_growth_cat, by=c("Federal_EIN" = "federal_ein")) %>%
    select(Person_ID, emp_rate_cat)

head(cohort_wages_merged_growth_cat)

## Merging

At this point, we have created a lookup table, **cohort_wages_merged_growth_cat**, which tracks those in the cohort who were employed at the time of their apprenticeship completion and the growth rate category of their primary employer at the time. Now, we can combine this information with that of future employment information for these individuals, which is saved in **cohort_wages_empr**. 
 
We can do so by leveraging the `inner_join` function so that we only include employment histories for individuals were employed in the quarter of completion and whose primary employers were tracked in our employer measures table in 2014. We perform this join on **ssn** and not a combination of **ssn** and **federal_ein** because we are interested in seeing *any* form of employment as opposed to looking at employment with the same employer. Our goal is to look at initial employment after graduation and how it affects our cohort's trajectories over the next year. 

In [ ]:
emp_empr_matches_growth <- cohort_wages_empr %>%
    inner_join(cohort_wages_merged_growth_cat, by="Person_ID") %>%
    select(Quarters_Relative_to_Completion, emp_rate_cat, Person_ID)

head(emp_empr_matches_growth)

We use the code below to save this table in our SQL database – we will utilize this for one of the later notebooks in Module 3. We have commented this out for now. 

    qry <- "use tr_ar_2022;"
    DBI::dbExecute(con, qry)

    DBI::dbWriteTable(
        conn = con,
        name = DBI::SQL("dbo.merged_cohort_wages_growth"), 
        value = emp_empr_matches_growth,
        overwrite = TRUE
    )


## Percentage of Cohort Employed

In this section, we will calculate the percentage of our cohort employed in each quarter by the type (high-growth/medium-growth/low-growth) of primary employer they were employed by after they completed an apprenticeship.

> Note: In the Firm Characteristics section, we looked at the number of quarters an individual worked at the same employer. Here, we are looking at each quarter and calculating the percentage of individuals within each group. 

In [ ]:
# count number of people within each emp_rate_cat and Quarters_Relative_to_Completion subgroup
# can use count() again because there is one row per person/quarter combination
df_counts <- emp_empr_matches_growth %>% 
    count(emp_rate_cat, Quarters_Relative_to_Completion)

df_counts

From here, we will create a mini-data frame that stores the initial counts of those employed in the quarter of apprenticeship completion based on the employment rate category of their primary employer.

In [ ]:
# store values for each emp_rate_cat group at Quarters_Relative_to_Competion = 0
first_ob <- df_counts %>% 
    filter(Quarters_Relative_to_Completion == 0) %>%
    # renaming "n" column to differentiate from that in df_counts
    rename(n_start = n) %>% 
    # don't need Quarters_Relative_to_Completion since it is filtered to 0
    select(-Quarters_Relative_to_Completion)

first_ob

Now, we can join this mini-data frame **first_ob** back to **df_counts** to calculate the percentage employed based on the employment rate categorization of their primary employer at the time of their apprenticeship completion.

In [ ]:
df_counts_prop <- df_counts %>%
    inner_join(first_ob, by="emp_rate_cat") %>%
    mutate(prop = n/n_start)

df_counts_prop

## Visualizing Results

Lastly, we visualize our results to understand how the employer growth category impacts trajectories of apprenticeship completers after they finish their training. 

In [ ]:
df_counts_prop %>%
    ggplot(aes(x = Quarters_Relative_to_Completion, y = prop, color = emp_rate_cat)) +  
    geom_line()

One might think that high growth rate jobs would lead to more REDACTED employment outcomes (at least in terms of an employment indicator), but this does not seem to be the case, especially in later quarters. In the figure above, you can see that those who were initially employed in high growth jobs are the REDACTED likely to be employed four quarters after their apprenticeship completion. It is likely that a lot of things correlated with these high growth employers. Additionally, note that we are working with just one year of apprenticeship completers, and further narrowed it to those (1) with employment at the time of apprenticeship completion and (2) were employed by a primary employer that existed in the employer measures file.

This is why instead of looking at measures in isolation, you might want to group employers based on multiple measures as seen in the next notebook: `04_Characterizing_Demand_Advanced.ipynb`. 

> Note: The next notebook (`04_Characterizing_Demand_Advanced.ipynb`) uses Unsupervised Machine Learning and more advanced coding, which might not be appropriate for all class participants.

## References

TDC Characterizing Demand, Beginners (more to come)